In [ ]:
import sqlite3

import pandas as pd


db = sqlite3.connect('../../data/amazon.db')

drive_df = pd.read_sql('select * from drive', db)
drive_df.head()

In [ ]:
def search(asin):
    drive_ts = pd.read_sql_query(f'select keepa_minute, value1 from drive_ts where asin = "{asin}" and keepa_idx=1 and value1 != -1', db)
    drive_ts['ts'] = pd.to_datetime((drive_ts['keepa_minute'] + 21564000) * 60, unit='s')
    drive_ts.set_index('ts', inplace=True)
    drive_ts = drive_ts.resample('1D').ffill()
    drive_ts = drive_ts.resample('M').mean()
    if '2021-01' not in drive_ts.index:
        return None
    base_price = drive_ts.loc['2021-01']['value1'][0]
    price_dict = {'base_price': base_price}
    for d in ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10']:
        if d in drive_ts.index:
            price_dict[d] = drive_ts.loc[d]['value1'][0] / base_price
        else:
            price_dict[d] = 1
    return price_dict

print(search(drive_df['asin'][15]))

In [ ]:
df_dict = []

for idx, row in drive_df.iterrows():
    ts = search(row['asin'])
    if ts is None:
        continue
    ts['asin'] = row['asin']
    ts['brand'] = row['brand']
    ts['product'] = row['model_name']
    ts['drive_type'] = row['drive_type']
    ts['drive_subtype'] = row['drive_subtype']
    ts['capacity'] = row['capacity']
    df_dict.append(ts)


df = pd.DataFrame(df_dict)
df.insert(0, 'CHIA_IMPACT', [0] * len(df))
len(df)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=60, random_state=501)
df_train.to_csv('drive_train.csv', index=False)
df_test.to_csv('drive_test.csv', index=False)
db.close()